In [83]:
import math
import numpy as np
import pandas as pd
import seaborn as sns
import os
import sklearn
from sklearn.metrics import precision_score, recall_score, roc_auc_score, matthews_corrcoef, balanced_accuracy_score, confusion_matrix, f1_score

from imblearn.under_sampling import RandomUnderSampler, NearMiss, TomekLinks, ClusterCentroids
from imblearn.ensemble import EasyEnsembleClassifier, BalancedBaggingClassifier
from imblearn.pipeline import Pipeline
from sklearn.datasets import make_classification
import sys

sys.path.append('/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/')
from RF_atomver import *

# adding CM and prediction types to all GP dataframes 

In [87]:
metric_cols = ['model',  'accuracy', 'precision', 'recall', 'specificity',
       'TN', 'FN', 'FP', 'TP', 'cm','ROC-AUC','MCC',
    'Balanced Accuracy','f1', 'NEK', 'feat_type', 'strategy']
def all_GP_metrics(file_path, filename): 
    df = pd.read_csv(file_path+filename)
    df['cm'] = confusion_matrix
    df['ROC-AUC'] = roc_auc_score(df['y'], df['y_pred'])
    df['MCC'] = matthews_corrcoef(df['y'], df['y_pred'])
    df['Balanced Accuracy']=balanced_accuracy_score(df['y'], df['y_pred'])
    df['f1']=f1_score(df['y'], df['y_pred'])
    # df.to_csv(file_path+ filename, index=False)
    results = df.iloc[[0]]
    results[metric_cols]
    results = results[metric_cols].values
    return results
    
    
    

In [85]:
testing1 = pd.read_csv('/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/NEK/GP/GP_results/NEK2_inhibition_moe_UNDER_test_GP.csv')
testing1

,mean_pred_class0,mean_pred_class1,y,y_pred,var_pred_class0,var_pred_class1,pred_prob_class0,pred_prob_class1,pred_prob_std_class0,pred_prob_std_class1,...,likelihood,observed_pred,accuracy,precision,recall,specificity,TN,FN,FP,TP
0,-3.432489,-3.559694,0,0,2.910149,3.408390,0.478955,0.521045,0.360817,0.360817,...,DirichletClassificationLikelihood(\n (noise_c...,"MultivariateNormal(loc: torch.Size([2, 409]))",0.865526,0.309859,0.785714,0.871391,332,6,49,22
1,-1.626352,-4.343420,0,0,2.896464,3.337575,0.796647,0.203353,0.256310,0.256310,...,DirichletClassificationLikelihood(\n (noise_c...,"MultivariateNormal(loc: torch.Size([2, 409]))",0.865526,0.309859,0.785714,0.871391,332,6,49,22
2,-0.326815,-5.800612,0,0,2.296502,2.753218,0.948136,0.051864,0.108677,0.108677,...,DirichletClassificationLikelihood(\n (noise_c...,"MultivariateNormal(loc: torch.Size([2, 409]))",0.865526,0.309859,0.785714,0.871391,332,6,49,22
3,-0.868721,-4.620207,0,0,1.636043,1.929262,0.932225,0.067774,0.100553,0.100553,...,DirichletClassificationLikelihood(\n (noise_c...,"MultivariateNormal(loc: torch.Size([2, 409]))",0.865526,0.309859,0.785714,0.871391,332,6,49,22
4,-0.538995,-6.026606,0,0,1.933182,2.620340,0.968881,0.031118,0.062041,0.062041,...,DirichletClassificationLikelihood(\n (noise_c...,"MultivariateNormal(loc: torch.Size([2, 409]))",0.865526,0.309859,0.785714,0.871391,332,6,49,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
404,-5.557435,-1.117711,1,1,4.133475,3.985746,0.057874,0.942126,0.126763,0.126763,...,DirichletClassificationLikelihood(\n (noise_c...,"MultivariateNormal(loc: torch.Size([2, 409]))",0.865526,0.309859,0.785714,0.871391,332,6,49,22
405,-6.450356,-0.306143,1,1,3.222914,2.773119,0.022402,0.977598,0.061156,0.061156,...,DirichletClassificationLikelihood(\n (noise_c...,"MultivariateNormal(loc: torch.Size([2, 409]))",0.865526,0.309859,0.785714,0.871391,332,6,49,22
406,-1.884431,-4.505947,1,0,6.701648,7.468300,0.763131,0.236869,0.317857,0.317857,...,DirichletClassificationLikelihood(\n (noise_c...,"MultivariateNormal(loc: torch.Size([2, 409]))",0.865526,0.309859,0.785714,0.871391,332,6,49,22
407,-4.472875,-1.780880,1,1,2.333416,2.336636,0.143699,0.856301,0.232741,0.232741,...,DirichletClassificationLikelihood(\n (noise_c...,"MultivariateNormal(loc: torch.Size([2, 409]))",0.865526,0.309859,0.785714,0.871391,332,6,49,22


In [88]:
data_dir = '/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/NEK/'
result_path  = '/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/NEK/GP/GP_results/'
nek_nums = [2,3,5,9]
samplings = ['scaled', 'UNDER', 'SMOTE', 'ADASYN'] 
features = ['moe', 'mfp']
NEK= 'NEK'
train_results = [] 
test_results = [] 
for i, n in enumerate(nek_nums):
    nek = str(n)
    # print(f'NEK{nek}')
    nek_path= f'{data_dir}NEK{nek}/bind/'
    for k, feat in enumerate(features): 
        print()
        for j, samp in enumerate(samplings): 
        
            print(f'NEK{nek} bind {feat} {samp}')
            file_root = f'NEK{nek}_binding_{feat}_{samp}'
       
            train = all_GP_metrics(result_path, file_root+'_train_GP.csv') 
            test = all_GP_metrics(result_path, file_root+'_test_GP.csv') 

            train_results.append(train)
            test_results.append(test) 
            
            
        
            print()

    
    if n == 2 or n == 9:

        nek_path= f'{data_dir}NEK{nek}/inhib/'
        for k, feat in enumerate(features): 
            print()
            for j, samp in enumerate(samplings): 
                print(f'NEK{nek} inhib {feat} {samp}')
                file_root = f'NEK{nek}_inhibition_{feat}_{samp}'
                print(f'NEK{nek} inbhib{feat} {samp}')
                train = all_GP_metrics(result_path, file_root+'_train_GP.csv') 
                test = all_GP_metrics(result_path, file_root+'_test_GP.csv') 
                train_results.append(train)
                test_results.append(test) 
    
        
            
    print()


NEK2 bind moe scaled


KeyError: "['cm', 'NEK', 'feat_type', 'strategy'] not in index"

In [69]:
# train_results = np.squeeze(train_results, axis=1)
# train_results

# train_df =  pd.DataFrame(train_results,columns=metric_cols)
# train_df

In [79]:
test_results = np.squeeze(test_results, axis=1)
test_results

test_df =  pd.DataFrame(test_results,columns=metric_cols)
test_df

,model,accuracy,precision,recall,specificity,TN,FN,FP,TP,cm,ROC-AUC,MCC,Balanced Accuracy,f1,NEK,feat_type,strategy
0,NEK2_binding_moe_scaled_GP,0.957597,0.0,0.0,1.0,271,12,0,0,"[271, 0, 12, 0]",0.5,0.0,0.5,0.0,NEK2_binding,moe,scaled
1,NEK2_binding_moe_UNDER_GP,0.738516,0.102564,0.666667,0.741697,201,4,70,8,"[201, 70, 4, 8]",0.704182,0.184161,0.704182,0.177778,NEK2_binding,moe,UNDER
2,NEK2_binding_moe_SMOTE_GP,0.968198,0.8,0.333333,0.99631,270,8,1,4,"[270, 1, 8, 4]",0.664822,0.504211,0.664822,0.470588,NEK2_binding,moe,SMOTE
3,NEK2_binding_moe_ADASYN_GP,0.961131,0.571429,0.333333,0.98893,268,8,3,4,"[268, 3, 8, 4]",0.661132,0.418102,0.661132,0.421053,NEK2_binding,moe,ADASYN
4,NEK2_binding_mfp_scaled_GP,0.957597,0.0,0.0,1.0,271,12,0,0,"[271, 0, 12, 0]",0.5,0.0,0.5,0.0,NEK2_binding,mfp,scaled
5,NEK2_binding_mfp_UNDER_GP,0.954064,0.0,0.0,0.99631,270,12,1,0,"[270, 1, 12, 0]",0.498155,-0.012531,0.498155,0.0,NEK2_binding,mfp,UNDER
6,NEK2_binding_mfp_SMOTE_GP,0.964664,1.0,0.166667,1.0,271,10,0,2,"[271, 0, 10, 2]",0.583333,0.400918,0.583333,0.285714,NEK2_binding,mfp,SMOTE
7,NEK2_binding_mfp_ADASYN_GP,0.964664,1.0,0.166667,1.0,271,10,0,2,"[271, 0, 10, 2]",0.583333,0.400918,0.583333,0.285714,NEK2_binding,mfp,ADASYN
8,NEK2_inhibition_moe_scaled_GP,0.93154,0.0,0.0,1.0,381,28,0,0,"[381, 0, 28, 0]",0.5,0.0,0.5,0.0,NEK2_inhibition,moe,scaled
9,NEK2_inhibition_moe_UNDER_GP,0.865526,0.309859,0.785714,0.871391,332,6,49,22,"[332, 49, 6, 22]",0.828553,0.438116,0.828553,0.444444,NEK2_inhibition,moe,UNDER


In [80]:
train_df.to_csv('GP_train_results_all_NEK.csv',index=False)
test_df.to_csv('GP_test_results_all_NEK.csv',index=False)

# Try to select the max and min for each of the metrics and display that model
For all models together

In [81]:
# max_cols = ['accuracy', 'precision', 'recall', 'specificity',
#        'TN', 'FN', 'FP', 'TP', 'ROC-AUC']
max_cols = ['accuracy', 'precision', 'recall', 'specificity',
       'TN',  'TP', 'ROC-AUC', 'MCC', 'Balanced Accuracy','f1']
min_cols = ['FN', 'FP']
max_rows = test_df.groupby('model')[max_cols].max()

max_rows

,accuracy,precision,recall,specificity,TN,TP,ROC-AUC,MCC,Balanced Accuracy,f1
model,,,,,,,,,,
NEK2_binding_mfp_ADASYN_GP,0.964664,1.0,0.166667,1.0,271,2,0.583333,0.400918,0.583333,0.285714
NEK2_binding_mfp_SMOTE_GP,0.964664,1.0,0.166667,1.0,271,2,0.583333,0.400918,0.583333,0.285714
NEK2_binding_mfp_UNDER_GP,0.954064,0.0,0.0,0.99631,270,0,0.498155,-0.012531,0.498155,0.0
NEK2_binding_mfp_scaled_GP,0.957597,0.0,0.0,1.0,271,0,0.5,0.0,0.5,0.0
NEK2_binding_moe_ADASYN_GP,0.961131,0.571429,0.333333,0.98893,268,4,0.661132,0.418102,0.661132,0.421053
NEK2_binding_moe_SMOTE_GP,0.968198,0.8,0.333333,0.99631,270,4,0.664822,0.504211,0.664822,0.470588
NEK2_binding_moe_UNDER_GP,0.738516,0.102564,0.666667,0.741697,201,8,0.704182,0.184161,0.704182,0.177778
NEK2_binding_moe_scaled_GP,0.957597,0.0,0.0,1.0,271,0,0.5,0.0,0.5,0.0
NEK2_inhibition_mfp_ADASYN_GP,0.96577,0.888889,0.571429,0.994751,379,16,0.78309,0.697061,0.78309,0.695652


In [82]:
min_cols = ['FN', 'FP']
min_rows = test_df.groupby('model')[min_cols].min()
min_rows

,FN,FP
model,,
NEK2_binding_mfp_ADASYN_GP,10,0
NEK2_binding_mfp_SMOTE_GP,10,0
NEK2_binding_mfp_UNDER_GP,12,1
NEK2_binding_mfp_scaled_GP,12,0
NEK2_binding_moe_ADASYN_GP,8,3
NEK2_binding_moe_SMOTE_GP,8,1
NEK2_binding_moe_UNDER_GP,4,70
NEK2_binding_moe_scaled_GP,12,0
NEK2_inhibition_mfp_ADASYN_GP,12,2


In [74]:
tntp = ['TN', 'TP']
tntp_rows = test_df.groupby('model')[tntp].max()
tntp_rows

,TN,TP
model,,
NEK2_binding_mfp_ADASYN_GP,271,2
NEK2_binding_mfp_SMOTE_GP,271,2
NEK2_binding_mfp_UNDER_GP,270,0
NEK2_binding_mfp_scaled_GP,271,0
NEK2_binding_moe_ADASYN_GP,268,4
NEK2_binding_moe_SMOTE_GP,270,4
NEK2_binding_moe_UNDER_GP,201,8
NEK2_binding_moe_scaled_GP,271,0
NEK2_inhibition_mfp_ADASYN_GP,379,16


# Displaying the max metric more clearly ? 
for all models together

In [11]:
for i, metric in enumerate(max_cols): 
    max_metric = test_df.loc[test_df[metric].idxmax()]
    print(f'{metric}:')
    display(max_metric)
    print()
print()
print('\nLowest FN and FP\n')
for i, metric in enumerate(min_cols): 
    min_metric = test_df.loc[test_df[metric].idxmax()]
    print(f'{metric}:')
    display(min_metric)
    print()

TypeError: reduction operation 'argmax' not allowed for this dtype

# Adding a 'NEK#' column 
so it's easier to compare across one type of nek

In [12]:
new_test = test_df
new_test['NEK'] = new_test['model'].str[0:4]
new_test

,model,accuracy,precision,recall,specificity,TN,FN,FP,TP,cm,ROC-AUC,MCC,Balanced Accuracy,f1,NEK
0,NEK2_binding_moe_scaled_GP,0.957597,0.0,0.0,1.0,271,12,0,0,"[271, 0, 12, 0]",0.5,0.0,0.5,0.0,NEK2
1,NEK2_binding_moe_UNDER_GP,0.738516,0.102564,0.666667,0.741697,201,4,70,8,"[201, 70, 4, 8]",0.704182,0.184161,0.704182,0.177778,NEK2
2,NEK2_binding_moe_SMOTE_GP,0.968198,0.8,0.333333,0.99631,270,8,1,4,"[270, 1, 8, 4]",0.664822,0.504211,0.664822,0.470588,NEK2
3,NEK2_binding_moe_ADASYN_GP,0.961131,0.571429,0.333333,0.98893,268,8,3,4,"[268, 3, 8, 4]",0.661132,0.418102,0.661132,0.421053,NEK2
4,NEK2_binding_mfp_scaled_GP,0.957597,0.0,0.0,1.0,271,12,0,0,"[271, 0, 12, 0]",0.5,0.0,0.5,0.0,NEK2
5,NEK2_binding_mfp_UNDER_GP,0.954064,0.0,0.0,0.99631,270,12,1,0,"[270, 1, 12, 0]",0.498155,-0.012531,0.498155,0.0,NEK2
6,NEK2_binding_mfp_SMOTE_GP,0.964664,1.0,0.166667,1.0,271,10,0,2,"[271, 0, 10, 2]",0.583333,0.400918,0.583333,0.285714,NEK2
7,NEK2_binding_mfp_ADASYN_GP,0.964664,1.0,0.166667,1.0,271,10,0,2,"[271, 0, 10, 2]",0.583333,0.400918,0.583333,0.285714,NEK2
8,NEK2_inhibition_moe_scaled_GP,0.93154,0.0,0.0,1.0,381,28,0,0,"[381, 0, 28, 0]",0.5,0.0,0.5,0.0,NEK2
9,NEK2_inhibition_moe_UNDER_GP,0.865526,0.309859,0.785714,0.871391,332,6,49,22,"[332, 49, 6, 22]",0.828553,0.438116,0.828553,0.444444,NEK2


In [13]:
neks = ['NEK2', 'NEK3', 'NEK5', 'NEK9']
for n, nek in enumerate(neks): 
    nek_df = new_test[new_test['NEK'] ==nek]
    print(nek)
    for i, metric in enumerate(max_cols): 
        max_metric = new_test.loc[new_test[metric].idxmax()]
        max_model_name = max_metric['model'] 
        print(f'{metric}:')
        # display(max_metric)
        print()
    print()
    print('\nLowest FN and FP\n')
    for j, metric in enumerate(min_cols): 
        min_metric = new_test.loc[new_test[metric].idxmax()]
        print(f'{metric}:')
        display(min_metric)
        print()


NEK2


TypeError: reduction operation 'argmax' not allowed for this dtype

In [21]:
df = pd.read_csv('/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/NEK/GP/GP_results/GP_test_results_all_NEK.csv')
df['NEK'] = df['model'].str[0:4]+'_'+train_df['model'].str.extract(r'_(.*?)_', expand=False)

df['strategy'] = df['model'].str[-10:-1].str.extract(r'_(.*?)_', expand=False)

df['feat_type'] =df['model'].str[-15:-1].str.extract(r'_(.*?)_', expand=False)
df['model_type'] = 'GP'
df.head(5)
df.to_csv('/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/NEK/GP/GP_results/GP_test_results_all_NEK.csv', index=False)



In [22]:
df.columns

Index(['model', 'accuracy', 'precision', 'recall', 'specificity', 'TN', 'FN',
       'FP', 'TP', 'cm', 'ROC-AUC', 'MCC', 'Balanced Accuracy', 'f1', 'NEK',
       'strategy', 'feat_type', 'model_type'],
      dtype='object')

In [16]:
list1 = list(df.columns)
list2 = ['recall', 'precision']
list3 = [m for m in list1 if m not in list2]
# list3

In [39]:
def rank(df, compare_type, metrics): 
    # metrics = ['recall', 'precision']
    # compare_type = 'NEK'

    if not isinstance(compare_type, list):
        compare_type = [compare_type]
    grouped = df.groupby(compare_type, group_keys=False).apply(lambda group: group.sort_values(by=metrics, ascending=[False]*len(metrics))) 
        # can do any lsit of metrics 
    cols = list(df.columns) 
    # keep_cols = ['model','model_type','NEK', 'feat_type','cm']
    keep_cols = ['model','cm']
    keep_cols.append(compare_type)
    keep_cols.extend(compare_type)
    keep_cols.append(metrics) 
    
    keep_cols.extend(metrics)
    # other_cols = [col for col in cols if col not in keep_cols]
    # print(cols)
    # print(other_cols)
    
    # ranked_df = grouped.reset_index(drop=True).drop(columns=other_cols)
    ranked_df = grouped.reset_index(drop=True)
    ranked_df
    return ranked_df

In [40]:
print(df['strategy'].dtype)

object


In [41]:
metrics = ['recall', 'precision','f1']
compare_types = ['NEK', 'strategy']

ranked_df = rank(df, compare_types, metrics)  
ranked_df 

,model,accuracy,precision,recall,specificity,TN,FN,FP,TP,cm,ROC-AUC,MCC,Balanced Accuracy,f1,NEK,strategy,feat_type,model_type
0,NEK2_binding_moe_ADASYN_GP,0.961131,0.571429,0.333333,0.988930,268,8,3,4,"[268, 3, 8, 4]",0.661132,0.418102,0.661132,0.421053,NEK2_binding,ADASYN,moe,GP
1,NEK2_binding_mfp_ADASYN_GP,0.964664,1.000000,0.166667,1.000000,271,10,0,2,"[271, 0, 10, 2]",0.583333,0.400918,0.583333,0.285714,NEK2_binding,ADASYN,mfp,GP
2,NEK2_binding_moe_SMOTE_GP,0.968198,0.800000,0.333333,0.996310,270,8,1,4,"[270, 1, 8, 4]",0.664822,0.504211,0.664822,0.470588,NEK2_binding,SMOTE,moe,GP
3,NEK2_binding_mfp_SMOTE_GP,0.964664,1.000000,0.166667,1.000000,271,10,0,2,"[271, 0, 10, 2]",0.583333,0.400918,0.583333,0.285714,NEK2_binding,SMOTE,mfp,GP
4,NEK2_binding_moe_UNDER_GP,0.738516,0.102564,0.666667,0.741697,201,4,70,8,"[201, 70, 4, 8]",0.704182,0.184161,0.704182,0.177778,NEK2_binding,UNDER,moe,GP
5,NEK2_binding_mfp_UNDER_GP,0.954064,0.000000,0.000000,0.996310,270,12,1,0,"[270, 1, 12, 0]",0.498155,-0.012531,0.498155,0.000000,NEK2_binding,UNDER,mfp,GP
6,NEK2_binding_moe_scaled_GP,0.957597,0.000000,0.000000,1.000000,271,12,0,0,"[271, 0, 12, 0]",0.500000,0.000000,0.500000,0.000000,NEK2_binding,scaled,moe,GP
7,NEK2_binding_mfp_scaled_GP,0.957597,0.000000,0.000000,1.000000,271,12,0,0,"[271, 0, 12, 0]",0.500000,0.000000,0.500000,0.000000,NEK2_binding,scaled,mfp,GP
8,NEK2_inhibition_mfp_ADASYN_GP,0.965770,0.888889,0.571429,0.994751,379,12,2,16,"[379, 2, 12, 16]",0.783090,0.697061,0.783090,0.695652,NEK2_inhibition,ADASYN,mfp,GP
9,NEK2_inhibition_moe_ADASYN_GP,0.951100,0.681818,0.535714,0.981627,374,13,7,15,"[374, 7, 13, 15]",0.758671,0.579098,0.758671,0.600000,NEK2_inhibition,ADASYN,moe,GP


In [48]:
nek2_bind = df[df['NEK'] == 'NEK2_binding']

compare_type =['feat_type']
metrics=['recall', 'ROC-AUC', 'f1']

nek2_ranked = rank(nek2_bind, compare_type,metrics)  
nek2_ranked

,model,accuracy,precision,recall,specificity,TN,FN,FP,TP,cm,ROC-AUC,MCC,Balanced Accuracy,f1,NEK,strategy,feat_type,model_type
0,NEK2_binding_mfp_SMOTE_GP,0.964664,1.000000,0.166667,1.000000,271,10,0,2,"[271, 0, 10, 2]",0.583333,0.400918,0.583333,0.285714,NEK2_binding,SMOTE,mfp,GP
1,NEK2_binding_mfp_ADASYN_GP,0.964664,1.000000,0.166667,1.000000,271,10,0,2,"[271, 0, 10, 2]",0.583333,0.400918,0.583333,0.285714,NEK2_binding,ADASYN,mfp,GP
2,NEK2_binding_mfp_scaled_GP,0.957597,0.000000,0.000000,1.000000,271,12,0,0,"[271, 0, 12, 0]",0.500000,0.000000,0.500000,0.000000,NEK2_binding,scaled,mfp,GP
3,NEK2_binding_mfp_UNDER_GP,0.954064,0.000000,0.000000,0.996310,270,12,1,0,"[270, 1, 12, 0]",0.498155,-0.012531,0.498155,0.000000,NEK2_binding,UNDER,mfp,GP
4,NEK2_binding_moe_UNDER_GP,0.738516,0.102564,0.666667,0.741697,201,4,70,8,"[201, 70, 4, 8]",0.704182,0.184161,0.704182,0.177778,NEK2_binding,UNDER,moe,GP
5,NEK2_binding_moe_SMOTE_GP,0.968198,0.800000,0.333333,0.996310,270,8,1,4,"[270, 1, 8, 4]",0.664822,0.504211,0.664822,0.470588,NEK2_binding,SMOTE,moe,GP
6,NEK2_binding_moe_ADASYN_GP,0.961131,0.571429,0.333333,0.988930,268,8,3,4,"[268, 3, 8, 4]",0.661132,0.418102,0.661132,0.421053,NEK2_binding,ADASYN,moe,GP
7,NEK2_binding_moe_scaled_GP,0.957597,0.000000,0.000000,1.000000,271,12,0,0,"[271, 0, 12, 0]",0.500000,0.000000,0.500000,0.000000,NEK2_binding,scaled,moe,GP


In [45]:
# df.columns

In [3]:
# def add_cm(filepath, filename): 
#     # print(filepath)
#     # print(filename)
#     df = pd.read_csv(filepath+filename)
    
#     true_labels = df['y'] 
#     predictions = df['y_pred']
#     cm = confusion_matrix(true_labels, predictions )
#     cm_flattened = cm.flatten().tolist()
#     df['cm'] = [cm_flattened]* len(df)
#     df['prediction_type'] = df.apply(lambda x: prediction_type(x['y'], x['y_pred']), axis=1)
#     return df
        

In [10]:
# df = pd.read_csv(result_path+ 'NEK2_binding_moe_scaled_test_GP.csv')
# true_labels = df['y'] 
# predictions = df['y_pred']
# cm = confusion_matrix(true_labels, predictions )
# df_copy = df.copy()
# cm_flattened = cm.flatten().tolist()
# cm_flattened
# # df_copy['cm'] =[(x,) for x in list(cm)]
# df_copy['cm'] = [cm_flattened]* len(df_copy)
# df_copy['prediction_type'] = df.apply(lambda x: prediction_type(x['y'], x['y_pred']), axis=1)

In [5]:
# data_dir = '/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/NEK/'
# gp_result_path = '/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/NEK/GP/GP_results/'
# # capstone/atom2024/atom2024/notebooks/NEK/GP/GP_results/NEK2_binding_mfp_ADASYN_test_GP.csv
# nek_nums = [2,3,5,9]
# samplings = ['scaled', 'UNDER', 'SMOTE', 'ADASYN'] 
# features = ['moe', 'mfp']
# NEK= 'NEK'
# for i, n in enumerate(nek_nums):
#     nek = str(n)
#     print(f'NEK{nek}')
#     nek_path= f'{data_dir}NEK{nek}/bind/'
    
#     for k, feat in enumerate(features): 
#         print()
#         for j, samp in enumerate(samplings): 
        
#             print(f'NEK{nek} {feat} {samp}')
#             file_root = f'NEK{nek}_binding_{feat}_{samp}'
#             train_file = f'{file_root}_train_GP.csv'
#             test_file = f'{file_root}_test_GP.csv'

#             train_df = add_cm(gp_result_path,train_file)
#             test_df = add_cm(gp_result_path,test_file)
#             train_df.to_csv(f'{gp_result_path}{file_root}_train_GP.csv',index=False) 
#             test_df.to_csv(f'{gp_result_path}{file_root}_test_GP.csv',index=False) 
            
#             print()
            
            
#     print()
    
#     if n == 2 or n == 9:

#         nek_path= f'{data_dir}NEK{nek}/inhib/'
#         for k, feat in enumerate(features): 
#             print()
#             for j, samp in enumerate(samplings): 
#                 file_root = f'NEK{nek}_inhibition_{feat}_{samp}'
#                 print(f'NEK{nek} {feat} {samp}')
#                 print(f'NEK{nek} {feat} {samp}')
#                 file_root = f'NEK{nek}_inhibition_{feat}_{samp}'
#                 train_file = f'{file_root}_train_GP.csv'
#                 test_file = f'{file_root}_test_GP.csv'
    
#                 train_df = add_cm(gp_result_path,train_file)
#                 test_df = add_cm(gp_result_path,test_file)
#                 train_df.to_csv(f'{gp_result_path}{file_root}_train_GP.csv',index=False) 
#                 test_df.to_csv(f'{gp_result_path}{file_root}_test_GP.csv',index=False) 
#                 print()
        
            
#     print()

NEK2

NEK2 moe scaled
/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/NEK/GP/GP_results/
NEK2_binding_moe_scaled_train_GP.csv
/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/NEK/GP/GP_results/
NEK2_binding_moe_scaled_test_GP.csv

NEK2 moe UNDER
/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/NEK/GP/GP_results/
NEK2_binding_moe_UNDER_train_GP.csv
/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/NEK/GP/GP_results/
NEK2_binding_moe_UNDER_test_GP.csv

NEK2 moe SMOTE
/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/NEK/GP/GP_results/
NEK2_binding_moe_SMOTE_train_GP.csv
/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/NEK/GP/GP_results/
NEK2_binding_moe_SMOTE_test_GP.csv

NEK2 moe ADASYN
/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/NEK/GP/GP_results/
NEK2_binding_moe_ADASYN_train_GP.csv
/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/NEK/GP/GP_results/
NEK2_binding_moe_ADASYN_test_GP.csv


NEK2 mf

In [8]:
# test = pd.read_csv('/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/NEK/GP/GP_results/NEK9_binding_mfp_ADASYN_test_GP.csv')